# FastGRNN and FastRNN in Keras

This is modified to Keras implementation of FastGRNN and FastRNN cells from a original simple notebook that illustrates the usage of tensorflow implementation. This is just only training result. The USPS dataset is used as with the original code.  
Original code is as follows.  
https://github.com/microsoft/EdgeML/tree/master/examples/tf/FastCells
  
Please refer to `fetch_usps.py` and run it for downloading and cleaning up the dataset.  

The original code license is as follows.  
Copyright (c) Microsoft Corporation. All rights reserved.  
Licensed under the MIT license.

In [1]:
import helpermethods
import tensorflow as tf
import numpy as np
import sys
import os
from tensorflow.keras.layers import Dense, Input, RNN
from tensorflow.keras.models import Model, Sequential, load_model
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import Adam

#Provide the GPU number to be used
os.environ['CUDA_VISIBLE_DEVICES'] =''

#FastRNN and FastGRNN imports
from rnn import FastGRNNCellKeras, FastRNNCellKeras

# Fixing seeds for reproducibility
tf.set_random_seed(42)
np.random.seed(42)

# USPS Data

--- Here is completely copied from the original code. ---  
  
  
It is assumed that the USPS data has already been downloaded and processed with [fetch_usps.py](fetch_usps.py) and [process_usps.py](process_usps.py), and is present in the `./usps10` subdirectory.

Note: Even though usps10 is not a time-series dataset, it can be assumed as, a time-series where each row is coming in at one single time.
So the number of timesteps = 16 and inputDims = 16

In [2]:
#Loading and Pre-processing dataset for FastCells
dataDir = "usps10"
(dataDimension, numClasses, Xtrain, Ytrain, Xtest, Ytest, mean, std) = helpermethods.preProcessData(dataDir)
print("Feature Dimension: ", dataDimension)
print("Num classes: ", numClasses)

Feature Dimension:  256
Num classes:  10


# Model Parameters

---Here is completely copied from the original code.---  
  

FastRNN and FastGRNN work for most of the hyper-parameters with which you could acheive decent accuracies on LSTM/GRU. Over and above that, you can use low-rank, sparsity and quatization to reduce model size upto 45x when compared to LSTM/GRU.

In [3]:
cell = "FastGRNN" # Choose between FastGRNN, FastRNN  ###, UGRNN, GRU and LSTM

inputDims = 16 #features taken in by RNN in one timestep
hiddenDims = 32 #hidden state of RNN

totalEpochs = 300
batchSize = 100

learningRate = 0.01
decayStep = 200
decayRate = 0.1

outFile = None #provide your file, if you need all the logging info in a file

#low-rank parameterisation for weight matrices. None => Full Rank
wRank = None 
uRank = None 

#Sparsity of the weight matrices. x => 100*x % are non-zeros
#Note: Sparsity inducing is not supported in this code. 
sW = 1.0 
sU = 1.0

#Non-linearities for the RNN architecture. Can choose from "tanh, sigmoid, relu, quantTanh, quantSigm"
update_non_linearity = "tanh"
gate_non_linearity = "sigmoid"

assert dataDimension % inputDims == 0, "Infeasible per step input, Timesteps have to be integer"

# FastCell Graph Object

In [4]:
#Create appropriate RNN cell object based on choice
if cell == "FastGRNN":
    FastCell = FastGRNNCellKeras(hiddenDims, gate_non_linearity=gate_non_linearity,
                            update_non_linearity=update_non_linearity,
                            wRank=wRank, uRank=uRank)
    #FastCell_1 = FastGRNNCellKeras(hiddenDims)
    #FastCell_2 = FastGRNNCellKeras(hiddenDims)
elif cell == "FastRNN":
    FastCell = FastRNNCellKeras(hiddenDims, update_non_linearity=update_non_linearity,
                           wRank=wRank, uRank=uRank)

    
###!!!Other Cells is not implemented for Keras!!!!###
elif cell == "UGRNN":
    FastCell = UGRNNLRCell(hiddenDims, update_non_linearity=update_non_linearity,
                           wRank=wRank, uRank=uRank)
elif cell == "GRU":
    FastCell = GRULRCell(hiddenDims, update_non_linearity=update_non_linearity,
                         wRank=wRank, uRank=uRank)
elif cell == "LSTM":
    FastCell = LSTMLRCell(hiddenDims, update_non_linearity=update_non_linearity,
                          wRank=wRank, uRank=uRank)
else:
    sys.exit('Exiting: No Such Cell as ' + cell)

# FastCell Trainer Model for Keras

In [7]:
x = inputs = Input(shape=[int(dataDimension / inputDims), inputDims], name='input')
layer_shape = K.int_shape(x)#x.get_shape()
print(layer_shape)
x = RNN(FastCell, return_sequences=False, name='rnn')(x)
#x = RNN(FastCell, return_sequences=True, name='rnn')(x)
#x = RNN(FastCell_1, return_sequences=True, name='rnn1')(x)
#x = RNN(FastCell_2, return_sequences=False, name='rnn2')(x)
out = Dense(numClasses, activation='softmax', name='dense')(x)
model = Model(inputs=inputs, outputs=out)
Adam_ = Adam(lr=0.01)
model.compile(optimizer=Adam_, loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()
#Y = tf.placeholder("float", [None, numClasses])

(None, 16, 16)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           (None, 16, 16)            0         
_________________________________________________________________
rnn (RNN)                    (None, 32)                3204      
_________________________________________________________________
dense (Dense)                (None, 10)                330       
Total params: 1,932
Trainable params: 1,932
Non-trainable params: 0
_________________________________________________________________


# Training

In [8]:
Xtrain_ = np.reshape(Xtrain, [-1, int(dataDimension / inputDims), inputDims])
Xtest_ = np.reshape(Xtest, [-1, int(dataDimension / inputDims), inputDims])
model.fit(Xtrain_, Ytrain, batchSize, epochs=totalEpochs, validation_data=(Xtest_, Ytest))

Train on 7291 samples, validate on 2007 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/300
7291/7291 [==============================] - 1s 154us/sample - loss: 0.9420 - acc: 0.6822 - val_loss: 0.6060 - val_acc: 0.8097
Epoch 2/300
7291/7291 [==============================] - 1s 78us/sample - loss: 0.3521 - acc: 0.8851 - val_loss: 0.4574 - val_acc: 0.8625
Epoch 3/300
7291/7291 [==============================] - 1s 82us/sample - loss: 0.2621 - acc: 0.9180 - val_loss: 0.3956 - val_acc: 0.8849
Epoch 4/300
7291/7291 [==============================] - 1s 84us/sample - loss: 0.2147 - acc: 0.9324 - val_loss: 0.3777 - val_acc: 0.8929
Epoch 5/300
7291/7291 [==============================] - 1s 74us/sample - loss: 0.1613 - acc: 0.9510 - val_loss: 0.3494 - val_acc: 0.8994
Epoch 6/300
7291/7291 [==============================] - 1s 75us/sample - loss: 0.1507 - acc: 0.9542 - val_loss: 0.3614 - val_acc: 0.8989
Epoch 7/300
7291/7291 [==============================] - 1s 84us/sample - l

7291/7291 [==============================] - 1s 75us/sample - loss: 0.0458 - acc: 0.9851 - val_loss: 0.4087 - val_acc: 0.9143
Epoch 58/300
7291/7291 [==============================] - 1s 78us/sample - loss: 0.0377 - acc: 0.9886 - val_loss: 0.3545 - val_acc: 0.9238
Epoch 59/300
7291/7291 [==============================] - 1s 82us/sample - loss: 0.0284 - acc: 0.9919 - val_loss: 0.3590 - val_acc: 0.9263
Epoch 60/300
7291/7291 [==============================] - 1s 82us/sample - loss: 0.0270 - acc: 0.9923 - val_loss: 0.3712 - val_acc: 0.9273
Epoch 61/300
7291/7291 [==============================] - 1s 81us/sample - loss: 0.0330 - acc: 0.9892 - val_loss: 0.3871 - val_acc: 0.9163
Epoch 62/300
7291/7291 [==============================] - 1s 80us/sample - loss: 0.0341 - acc: 0.9896 - val_loss: 0.4023 - val_acc: 0.9223
Epoch 63/300
7291/7291 [==============================] - 1s 78us/sample - loss: 0.0239 - acc: 0.9927 - val_loss: 0.3722 - val_acc: 0.9258
Epoch 64/300
7291/7291 [================

Epoch 116/300
7291/7291 [==============================] - 1s 81us/sample - loss: 0.0348 - acc: 0.9888 - val_loss: 0.4067 - val_acc: 0.9188
Epoch 117/300
7291/7291 [==============================] - 1s 83us/sample - loss: 0.0183 - acc: 0.9947 - val_loss: 0.3747 - val_acc: 0.9287
Epoch 118/300
7291/7291 [==============================] - 1s 84us/sample - loss: 0.0153 - acc: 0.9956 - val_loss: 0.4371 - val_acc: 0.9248
Epoch 119/300
7291/7291 [==============================] - 1s 83us/sample - loss: 0.0130 - acc: 0.9966 - val_loss: 0.4060 - val_acc: 0.9327
Epoch 120/300
7291/7291 [==============================] - 1s 82us/sample - loss: 0.0097 - acc: 0.9978 - val_loss: 0.4247 - val_acc: 0.9287
Epoch 121/300
7291/7291 [==============================] - 1s 82us/sample - loss: 0.0120 - acc: 0.9967 - val_loss: 0.4595 - val_acc: 0.9248
Epoch 122/300
7291/7291 [==============================] - 1s 83us/sample - loss: 0.0096 - acc: 0.9979 - val_loss: 0.4107 - val_acc: 0.9337
Epoch 123/300
7291/7

7291/7291 [==============================] - 0s 68us/sample - loss: 0.0023 - acc: 0.9997 - val_loss: 0.4311 - val_acc: 0.9273
Epoch 175/300
7291/7291 [==============================] - 1s 79us/sample - loss: 0.0029 - acc: 0.9999 - val_loss: 0.4317 - val_acc: 0.9273
Epoch 176/300
7291/7291 [==============================] - 1s 80us/sample - loss: 0.0032 - acc: 0.9996 - val_loss: 0.4323 - val_acc: 0.9278
Epoch 177/300
7291/7291 [==============================] - 1s 79us/sample - loss: 0.0310 - acc: 0.9900 - val_loss: 0.4398 - val_acc: 0.9198
Epoch 178/300
7291/7291 [==============================] - 1s 80us/sample - loss: 0.0554 - acc: 0.9826 - val_loss: 0.5449 - val_acc: 0.9008
Epoch 179/300
7291/7291 [==============================] - 1s 79us/sample - loss: 0.0677 - acc: 0.9789 - val_loss: 0.4584 - val_acc: 0.9218
Epoch 180/300
7291/7291 [==============================] - 1s 80us/sample - loss: 0.0459 - acc: 0.9835 - val_loss: 0.4213 - val_acc: 0.9253
Epoch 181/300
7291/7291 [=========

7291/7291 [==============================] - 1s 82us/sample - loss: 0.0106 - acc: 0.9968 - val_loss: 0.4532 - val_acc: 0.9297
Epoch 233/300
7291/7291 [==============================] - 1s 86us/sample - loss: 0.0122 - acc: 0.9960 - val_loss: 0.4597 - val_acc: 0.9263
Epoch 234/300
7291/7291 [==============================] - 1s 86us/sample - loss: 0.0090 - acc: 0.9979 - val_loss: 0.4400 - val_acc: 0.9283
Epoch 235/300
7291/7291 [==============================] - 1s 86us/sample - loss: 0.0071 - acc: 0.9981 - val_loss: 0.4634 - val_acc: 0.9312
Epoch 236/300
7291/7291 [==============================] - 1s 86us/sample - loss: 0.0174 - acc: 0.9952 - val_loss: 0.4433 - val_acc: 0.9292
Epoch 237/300
7291/7291 [==============================] - 1s 83us/sample - loss: 0.0206 - acc: 0.9929 - val_loss: 0.4555 - val_acc: 0.9287
Epoch 238/300
7291/7291 [==============================] - 1s 81us/sample - loss: 0.0270 - acc: 0.9908 - val_loss: 0.4663 - val_acc: 0.9248
Epoch 239/300
7291/7291 [=========

7291/7291 [==============================] - 1s 95us/sample - loss: 0.0124 - acc: 0.9959 - val_loss: 0.4170 - val_acc: 0.9297
Epoch 291/300
7291/7291 [==============================] - 1s 92us/sample - loss: 0.0114 - acc: 0.9966 - val_loss: 0.4105 - val_acc: 0.9362
Epoch 292/300
7291/7291 [==============================] - 1s 125us/sample - loss: 0.0124 - acc: 0.9966 - val_loss: 0.4623 - val_acc: 0.9292
Epoch 293/300
7291/7291 [==============================] - 1s 80us/sample - loss: 0.0151 - acc: 0.9963 - val_loss: 0.4710 - val_acc: 0.9317
Epoch 294/300
7291/7291 [==============================] - 1s 130us/sample - loss: 0.0125 - acc: 0.9957 - val_loss: 0.4414 - val_acc: 0.9317
Epoch 295/300
7291/7291 [==============================] - 1s 92us/sample - loss: 0.0220 - acc: 0.9927 - val_loss: 0.4882 - val_acc: 0.9228
Epoch 296/300
7291/7291 [==============================] - 1s 91us/sample - loss: 0.0280 - acc: 0.9908 - val_loss: 0.4380 - val_acc: 0.9258
Epoch 297/300
7291/7291 [=======